## Imports and Functions

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt

import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)


colors = pc.qualitative.D3
angstrom = "\u212B"
gamma = "\u0393"

In [ ]:
def generate_FFT_filtered_dataset(  # filter_functions.py
    theta, energy, phi, data, fp, fn, int_range=0.0, overwrite=False, new_fn=None
):
    new_fn = (
        new_fn
        if new_fn
        else f"{os.path.splitext(fn)[0]}_filteredFFT_{int_range:.2f}int.h5"
    )
    new_fn = os.path.join(fp, new_fn)
    if not overwrite and os.path.exists(new_fn):
        raise FileExistsError(f"{new_fn} already exists")
    new_data = []
    for p in phi:
        val = p
        xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(
            x=theta,
            y=energy,
            z=phi,
            data=data,
            slice_dim="z",
            slice_val=val,
            int_range=int_range,
        )
        fft_data = filter_functions.fft2d_mask(dataslice, plot=False)
        new_data.append(fft_data)
    new_data = np.array(new_data).T
    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data
        axes_names = [
            "theta",
            "energy",
            "phi",
        ]  # Change these to match your axes labels
        axes = [theta, energy, phi]
        for axis, name in zip(axes, axes_names):
            f[name] = axis
    return new_fn

# Convert Streak&K-Corrected Data to .h5

In [ ]:
# ## Convert Data ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\k_corrected"

# # STEP 1 ##
# # Convert ibw to hdf5
# fn = "FS2FS4_FFTstreakCor_gkw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, kx, ky, energy = HDF5_loader.load_hdf5(
#     ddir, "FS2FS4_FFTstreakCor_gkw.h5"
# )  # load data from hdf5
# data.shape, kx.shape, ky.shape, energy.shape

In [ ]:
# ## Convert Data ##

# ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\k_corrected"

# # STEP 1 ##
# # Convert ibw to hdf5
# fn = "FS3avg_FFTstreakCor_gkw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, kx, ky, energy = HDF5_loader.load_hdf5(
#     ddir, "FS3avg_FFTstreakCor_gkw.h5"
# )  # load data from hdf5
# data.shape, kx.shape, ky.shape, energy.shape

In [ ]:
# ## FFT filter datasets ##

# # Load original dataset
# fp = r"E:/atully/arpes_data/2022_June/ARPES/k_corrected"
# # fn = r"FS_HOMO_Lamp_avg_gkw.h5"
# fn = r"FS_HOMO1_avg_gkw_v2.h5"
# data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

# # Generate FFT filtered dataset
# generate_FFT_filtered_dataset(
#     theta=theta, energy=energy, phi=phi, data=data, fp=fp, fn=fn
# )

# Load Data

In [ ]:
## 5-6 ML Film (lamp): HOMO ##

ddir = r"E:/atully/arpes_data/2022_June/ARPES/k_corrected"
files = []
files = ["FS_HOMO_Lamp_avg_gkw_filteredFFT_0.00int.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_homo = ARPES_DATA[files[0]]

In [ ]:
## 5-6 ML Film (lamp): HOMO-1 ##

ddir = r"E:/atully/arpes_data/2022_June/ARPES/k_corrected"
files = []
files = ["FS_HOMO1_avg_gkw_v2_filteredFFT_0.00int.h5"]

ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, theta, phi_or_time, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=theta, phi_or_time=phi_or_time, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_homo1 = ARPES_DATA[files[0]]

## Gamma - K

In [ ]:
## Initial params ##
slice_dim = "z"
EF = 16.9  # lamp, June 2022
homo = (
    -2.05 + EF
)  # HOMO is at 2.05 eV below EF, based on fits from this data averaged with fits from tr-ARPES results

slice_val = 0
int_range = 0.1

x_bin = 1
y_bin = 1

xlim = None
ylim = None

In [ ]:
## HOMO ##

# xlim = (-0.65, 0.59)
# xlim = None
# ylim = None

## Limits for G - K
xlim = (-0.52, 0.025)  # matches left side limit on FS scans
ylim = (-2.77 + EF, -1.35 + EF)
#####

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_homo,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

## Plot data
fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> HOMO (k<sub>y</sub> = {slice_val})",
    xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]",
    yaxis_title=f"E - E<sub>F</sub> [eV]",
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(go.Heatmap(x=x_2d, y=y_2d - EF, z=d_2d, coloraxis="coloraxis"))

# fig.update_coloraxes(cmin=0, cmax=0.5)

fig.show()

x_homo, y_homo, d_homo = x_2d, y_2d, d_2d

In [ ]:
## HOMO-1 ##

# xlim = (-0.65, 0.59)
# xlim = None
# ylim = None


## Limits for G - K
ylim = (-4.03 + EF, -2.65 + EF)
xlim = (
    -0.52,
    0.025,
)  # matches left side limit on FS scans; small overlap for stitching purposes
#####

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_homo1,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

## Plot data
fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> HOMO-1 (k<sub>y</sub> = {slice_val})",
    xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]",
    yaxis_title=f"E - E<sub>F</sub> [eV]",
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(go.Heatmap(x=x_2d, y=y_2d - EF, z=d_2d, coloraxis="coloraxis"))

# fig.update_coloraxes(cmin=0, cmax=0.5)

fig.show()

x_homo1, y_homo1, d_homo1 = x_2d, y_2d, d_2d

In [ ]:
## Linearly interpolate homo data to match resolution of homo-1 data (xaxis) ##

print(d_homo.shape, d_homo1.shape)

x, y, d = x_homo, y_homo, d_homo

new_d = tr_functions.interpolate_dataset(x, y, d, xref=x_homo1)

fig = tr_functions.default_fig()
fig.add_trace(go.Heatmap(x=x_homo1, y=y, z=new_d))
# fig.show()

print(new_d.shape)

In [ ]:
## Stitch Data ##

x_s1, y_s1, data_s1 = tr_functions.stitch_2_datasets(
    new_d, x_homo1, y_homo, d_homo1, x_homo1, y_homo1, stitch_dim="y"
)

In [ ]:
## Plot Stitched Data ##

homo_zero = False
# homo_zero = True

if homo_zero:
    y_plot = y_s1 - homo
    # yaxis_title = "$E - E_{HOMO} \; [eV]$"
    yaxis_title = "E - E<sub>HOMO</sub> (eV)"
else:
    y_plot = y_s1 - EF
    yaxis_title = "E - E<sub>F</sub> (eV)"

fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure (k<sub>y</sub> = {slice_val})",
    xaxis_title=f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    height=600,
    width=900,
    # dtick_y=0.2,
)

fig.add_trace(
    go.Heatmap(
        x=x_s1,
        y=y_plot,
        z=analysis_functions.norm_data(data_s1),
        # z=data_s1,
        coloraxis="coloraxis",
    )
)

# gm = -0.375
gk = 0.43

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[-gk, 0, gk],
        ticktext=[f"K", f"{gamma}", f"K"],
    ),
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)

# fig.update_coloraxes(cmin=0, cmax=0.5)

# fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
x_gk, y_gk, data_gk = x_s1, y_plot, analysis_functions.norm_data(data_s1)

## Gamma - M

In [ ]:
## Initial params ##
slice_dim = "x"
EF = 16.9  # lamp
homo = (
    -2.05 + EF
)  # HOMO is at 2.05 eV below EF, based on fits from this data averaged with fits from tr-ARPES results

slice_val = 0
int_range = 0.1

x_bin = 1
y_bin = 1

xlim = None
ylim = None

In [ ]:
## HOMO ##

# xlim = (-1.3, 0.18)
# xlim = (-1.3, 0.025)  # small overlap for stitching purposes
xlim = (-1.07, 0.025)  # small overlap for stitching purposes
# ylim = (-2.72 + EF, -1.35 + EF)
ylim = (-2.8 + EF, -1.35 + EF)

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_homo,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

## Plot data
fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> HOMO (k<sub>x</sub> = {slice_val})",
    xaxis_title=f"k<sub>y</sub> [{angstrom}<sup>-1</sup>]",
    yaxis_title=f"E - E<sub>F</sub> [eV]",
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(go.Heatmap(x=x_2d, y=y_2d - EF, z=d_2d, coloraxis="coloraxis"))

# fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()

x_homo, y_homo, d_homo = x_2d, y_2d, d_2d

In [ ]:
## HOMO-1 ##

# xlim = (-1.3, 0.18)
# xlim = (-1.3, 0.025)  # small overlap for stitching purposes
xlim = (-1.07, 0.025)  # small overlap for stitching purposes
# ylim = (-4.06 + EF, -2.69 + EF)
ylim = (-4.03 + EF, -2.75 + EF)

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad_homo1,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

## Plot data
fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> HOMO-1 (k<sub>x</sub> = {slice_val})",
    xaxis_title=f"k<sub>y</sub> [{angstrom}<sup>-1</sup>]",
    yaxis_title=f"E - E<sub>F</sub> [eV]",
    # yaxis_title="$E_K \; [eV]$",
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(go.Heatmap(x=x_2d, y=y_2d - EF, z=d_2d, coloraxis="coloraxis"))

fig.show()

x_homo1, y_homo1, d_homo1 = x_2d, y_2d, d_2d

In [ ]:
## Fix Incorred Traces ##

## Step 1: Find index of bad trace by plotting without y-axis

fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure (k<sub>x</sub> = {slice_val})",
    xaxis_title=f"k<sub>y</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    height=500,
    width=500,
)

fig.add_trace(
    go.Heatmap(
        x=-1 * x_homo,
        # y=y_homo - EF,
        z=analysis_functions.norm_data(d_homo),
        coloraxis="coloraxis",
    )
)

gm = 0.375

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0, gm, gm * 2, gm * 3],
        ticktext=[f"{gamma}", f"M", f"{gamma}'", f"M'"],
    ),
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()

In [ ]:
# ## Step 2:

# # Bad traces (rows): 564

# # Set d_homo row 564 = 0.5 * (563 + 565)


def fix_row(data, row):
    row_below = data[row - 1]
    row_above = data[row + 1]
    data[row] = 0.5 * (row_below + row_above)


# fix_row(d_homo, 564)

In [ ]:
## Linearly interpolate homo data to match resolution of homo-1 data (xaxis) ##

print(d_homo.shape, d_homo1.shape)

x, y, d = x_homo, y_homo, d_homo

new_d = tr_functions.interpolate_dataset(x, y, d, xref=x_homo1)

fig = tr_functions.default_fig()
fig.add_trace(go.Heatmap(x=x_homo1, y=y, z=new_d))
# fig.show()

print(new_d.shape)

In [ ]:
## Stitch Data ##

x_s1, y_s1, data_s1 = tr_functions.stitch_2_datasets(
    new_d, x_homo1, y_homo, d_homo1, x_homo1, y_homo1, stitch_dim="y"
)

In [ ]:
fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure (k<sub>x</sub> = {slice_val})",
    xaxis_title=f"k<sub>y</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    height=600,
    width=900,
    # dtick_y=0.2,
)

fig.add_trace(
    go.Heatmap(
        x=-1 * x_homo1,
        y=y_homo - EF,
        z=analysis_functions.norm_data(new_d),
        coloraxis="coloraxis",
    )
)

gm = 0.375

# for v in [0, gm * 1, gm * 2, gm * 3]:
#     fig.add_vline(x=v, line_width=3, line_dash="dash", line_color="yellow")


fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0, gm, gm * 2, gm * 3],
        ticktext=[f"{gamma}", f"M", f"{gamma}'", f"M'"],
    ),
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)

# fig.update_coloraxes(cmin=0, cmax=0.8)

# fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure (k<sub>x</sub> = {slice_val})",
    xaxis_title=f"k<sub>y</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    height=600,
    width=900,
    # dtick_y=0.2,
)

fig.add_trace(
    go.Heatmap(
        x=-1 * x_homo1,
        y=y_homo1 - EF,
        z=analysis_functions.norm_data(d_homo1),
        coloraxis="coloraxis",
    )
)

gm = 0.375

# for v in [0, gm * 1, gm * 2, gm * 3]:
#     fig.add_vline(x=v, line_width=3, line_dash="dash", line_color="yellow")


fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0, gm, gm * 2, gm * 3],
        ticktext=[f"{gamma}", f"M", f"{gamma}'", f"M'"],
    ),
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)

# fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
## Plot Stitched Data ##

homo_zero = False
# homo_zero = True

if homo_zero:
    y_plot = y_s1 - homo
    # yaxis_title = "$E - E_{HOMO} \; [eV]$"
    yaxis_title = "E - E<sub>HOMO</sub> (eV)"
else:
    y_plot = y_s1 - EF
    yaxis_title = "E - E<sub>F</sub> (eV)"

fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure (k<sub>x</sub> = {slice_val})",
    xaxis_title=f"k<sub>y</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    height=600,
    width=900,
    # dtick_y=0.2,
)

# fig.add_trace(
#     go.Heatmap(
#         x=-1 * x_homo1,
#         y=y_homo,
#         z=analysis_functions.norm_data(new_d),
#         coloraxis="coloraxis",
#     )
# )

# fig.add_trace(
#     go.Heatmap(
#         x=-1 * x_homo1,
#         y=y_homo1,
#         z=analysis_functions.norm_data(d_homo1),
#         coloraxis="coloraxis",
#     )
# )


fig.add_trace(
    go.Heatmap(
        x=-1 * x_s1,
        y=y_plot,
        z=analysis_functions.norm_data(data_s1),
        coloraxis="coloraxis",
    )
)

# for v in [0, -0.36 * 1, -0.36 * 2, -0.36 * 3]:
gm = 0.375

# for v in [0, gm * 1, gm * 2, gm * 3]:
#     fig.add_vline(x=v, line_width=3, line_dash="dash", line_color="yellow")


fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[0, gm, gm * 2, gm * 3],
        ticktext=[f"{gamma}", f"M", f"{gamma}'", f"M'"],
    ),
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)

# fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
x_gm, y_gm, data_gm = -1 * x_s1, y_plot, analysis_functions.norm_data(data_s1)

## Combine Gamma - M and Gamma - K

In [ ]:
## Stitch Data ##

x_s2, y_s2, data_s2 = tr_functions.stitch_2_datasets(
    data_gk, x_gk, y_gk, data_gm, x_gm, y_gm, stitch_dim="x"
)

In [ ]:
## Plot Stitched Data ##

homo_zero = False
# homo_zero = True

if homo_zero:
    y_plot = y_s2 + 2.05
    # yaxis_title = "$E - E_{HOMO} \; [eV]$"
    yaxis_title = "E - E<sub>HOMO</sub> (eV)"
else:
    y_plot = y_s2
    yaxis_title = "E - E<sub>F</sub> (eV)"

fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure",
    xaxis_title=f"k<sub>||</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    height=665,
    width=580,
    # dtick_y=0.2,
)

fig.add_trace(
    go.Heatmap(
        x=x_s2,
        y=y_plot,
        z=analysis_functions.norm_data(data_s2),
        coloraxis="coloraxis",
    )
)

gm = 0.375
gk = 0.43

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[-gk, 0, gm, gm * 2, gm * 3],
        ticktext=[f"K", f"{gamma}", f"M", f"{gamma}'", f"M'"],
    ),
    yaxis=dict(
        tickmode="array",
        tickvals=[-1.7, -2.1, -2.5, -2.9, -3.3, -3.8],
    ),
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)

# fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
## Plot Stitched Data ##

homo_zero = False
# homo_zero = True

if homo_zero:
    y_plot = y_s2 + 2.05
    # yaxis_title = "$E - E_{HOMO} \; [eV]$"
    yaxis_title = "E - E<sub>HOMO</sub> (eV)"
else:
    y_plot = y_s2
    yaxis_title = "E - E<sub>F</sub> (eV)"

fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure",
    xaxis_title=f"k<sub>||</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    # height=600,
    # width=900,
    height=500,
    width=500,
    # dtick_y=0.2,
)

fig.add_trace(
    go.Heatmap(
        x=x_s2,
        # y=y_plot,
        z=analysis_functions.norm_data(data_s2),
        coloraxis="coloraxis",
    )
)

gm = 0.375
gk = 0.43

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[-gk, 0, gm, gm * 2, gm * 3],
        ticktext=[f"K", f"{gamma}", f"M", f"{gamma}'", f"M'"],
    ),
    # yaxis=dict(
    #     tickmode="array",
    #     tickvals=[-1.7, -2.1, -2.5, -2.9, -3.3, -3.8],
    # ),
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)

# fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
## Fix indices (641) 640-642 and (156) 155-157


def fix_3rows(data, row):
    row_2below = data[row - 2]
    # row_below = data[row - 1]
    # row_above = data[row + 1]
    row_2above = data[row + 2]
    data[row] = 0.5 * (row_2below + row_2above)
    data[row - 1] = 0.5 * (row_2below + row_2above)
    data[row + 1] = 0.5 * (row_2below + row_2above)


fix_3rows(data_s2, 641)
fix_3rows(data_s2, 155)
fix_row(data_s2, 267)

In [ ]:
## Plot Stitched Data ##

homo_zero = False
# homo_zero = True

if homo_zero:
    y_plot = y_s2 + 2.05
    # yaxis_title = "$E - E_{HOMO} \; [eV]$"
    yaxis_title = "E - E<sub>HOMO</sub> (eV)"
else:
    y_plot = y_s2
    yaxis_title = "E - E<sub>F</sub> (eV)"

fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure",
    xaxis_title=f"k<sub>||</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    height=665,
    width=580,
    # height=500,
    # width=500,
    # dtick_y=0.2,
)

fig.add_trace(
    go.Heatmap(
        x=x_s2,
        # y=y_plot,
        z=analysis_functions.norm_data(data_s2),
        coloraxis="coloraxis",
    )
)

gm = 0.375
gk = 0.43

fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[-gk, 0, gm, gm * 2, gm * 3],
        ticktext=[f"K", f"{gamma}", f"M", f"{gamma}'", f"M'"],
    ),
    yaxis=dict(
        tickmode="array",
        tickvals=[-1.7, -2.1, -2.5, -2.9, -3.3, -3.8],
    ),
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)

# fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of C<sub>60</sub> HOMO and HOMO-1",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_s2, y=y_s2, data=analysis_functions.norm_data(data_s2), ylims=None, x_range=None
)

# Plot Data
fig.add_trace(go.Scatter(x=y_plot, y=col, name="data", line=dict(color=colors[0])))

fig.show()

In [ ]:
## Fit Data ##

x = y_plot
data = col

offset_type = "constant"

## Offset
# c = np.mean(data)
c = 0
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## Gaussian
if homo_zero == True:
    gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=0, sigma=0.5)
    gauss2 = fitting_functions.make_gaussian(
        num="B_", amplitude=1, center=-1.3, sigma=0.5
    )

else:
    gauss1 = fitting_functions.make_gaussian(
        num="A_", amplitude=1, center=-2, sigma=0.5
    )
    gauss2 = fitting_functions.make_gaussian(
        num="B_", amplitude=1, center=-3.3, sigma=0.5
    )


# ## Lorentzian
# lorentz1 = fitting_functions.make_lorentzian(num="A_", amplitude=1, center=0, sigma=0.5)
# lorentz2 = fitting_functions.make_lorentzian(
#     num="B_", amplitude=1, center=-1.3, sigma=0.5
# )


full_model = gauss1 + gauss2 + offset
# full_model = lorentz1 + lorentz2 + offset

fit = full_model.fit(data, x=x)

fit.plot()

print("HOMO")
print(fit.params["iA__center"])
print(fit.params["iA__fwhm"])
print("HOMO-1")
print(fit.params["iB__center"])
print(fit.params["iB__fwhm"])

# fit.params

In [ ]:
## Plot Fit on Data ##

components = fit.eval_components(x=y_plot)

fig = tr_functions.thesis_fig(
    title=f"Data + Fit Components",
    # xaxis_title=yaxis_title,
    xaxis_title="Intensity (arb. u)",
    yaxis_title=yaxis_title,
    equiv_axes=False,
    gridlines=False,
    # dtick_y=0.2,
    height=650,
    width=400,
)


fig.add_trace(
    go.Scatter(
        x=col,
        y=y_plot,
        name=f"data",
        line=dict(color=colors[0]),
    )
)

# fig.add_trace(go.Scatter(x=y_plot, y=fit.eval(x=y_plot), name="fit"))

# for model_name, model_value in list(components.items())[0:2]:
#     fig.add_annotation(
#         x=fit.params[f"{model_name}center"].value,
#         # y=fit.eval(x=fit.params[f"{model_name}center"].value),
#         # xref="x domain",
#         yref="y domain",
#         # The arrow head will be 25% along the x axis, starting from the left
#         # x=0.25,
#         # The arrow head will be 40% along the y axis, starting from the bottom
#         y=0.1,
#         showarrow=False,
#         text=f'Center: {fit.params[f"{model_name}center"].value:.2f} eV<br>FWHM: {fit.params[f"{model_name}fwhm"].value:.2f} eV',
#         font=dict(size=18, color=colors[0]),
#         # ax=-300,
#         # ay=0,
#         bgcolor="white",
#         # opacity=1,
#         # bordercolor=colors[0],
#         # borderwidth=2,
#         # borderpad=4,
#     )

# fig.show()

for model_name, model_value in list(components.items())[0:2]:
    fig.add_annotation(
        y=fit.params[f"{model_name}center"].value,
        # x=fit.eval(x=fit.params[f"{model_name}center"].value),
        xref="x domain",
        # yref="y domain",
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        # y=0.1,
        showarrow=False,
        text=f'Center: {fit.params[f"{model_name}center"].value:.2f} eV<br>FWHM: {fit.params[f"{model_name}fwhm"].value:.2f} eV',
        font=dict(size=16, color=colors[0]),
        # ax=-300,
        # ay=0,
        bgcolor="white",
        # opacity=1,
        # bordercolor=colors[0],
        # borderwidth=2,
        # borderpad=4,
    )

fig.show()

In [ ]:
## Plot Fit Components on Data ##

components = fit.eval_components(x=y_plot)

fig = tr_functions.thesis_fig(
    title=f"Data + Fit Components",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.2,
)

fig.add_trace(
    go.Scatter(
        x=y_plot,
        y=col - components["parabolic"],
        name=f"data",
        line=dict(color=colors[0]),
    )
)

for offset, (model_name, model_value) in zip([0, 0], list(components.items())[0:2]):
    fig.add_trace(
        go.Scatter(
            x=y_plot,
            y=model_value + offset,
            name=model_name,
        )
    )

fig.show()

In [ ]:
## Plot waterfall plot of 3 blobs ##
window = 0.05
colors = pc.qualitative.D3

homo_zero = False
homo_zero = True

if homo_zero:
    y_plot = y_s2 + 2.05
    # yaxis_title = "$E - E_{HOMO} \; [eV]$"
    yaxis_title = "E - E<sub>HOMO</sub> (eV)"
else:
    y_plot = y_s2
    yaxis_title = "E - E<sub>F</sub> (eV)"


fig1 = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> Band Structure",
    xaxis_title=f"k<sub>||</sub> ({angstrom}<sup>-1</sup>)",
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=False,
    height=600,
    width=900,
)

fig1.add_trace(
    go.Heatmap(
        x=x_s2,
        y=y_plot,
        z=analysis_functions.norm_data(data_s2),
        coloraxis="coloraxis",
    )
)


spacing = 1.2  # waterfall plot

a = 0.375
b = 0.43

val = [b * -1, 0, a * 1, a * 2, a * 3]

xlim1 = (val[0] - window, val[0] + window)
xlim2 = (val[1] - window, val[1] + window)
xlim3 = (val[2] - window, val[2] + window)
xlim4 = (val[3] - window, val[3] + window)
xlim5 = (val[4] - window, val[4] + window)

xlims = [xlim1, xlim2, xlim3, xlim4, xlim5]

names = [f"K", f"{gamma}", f"M", f"{gamma}'", f"M'"]

## Plot data
# fig = tr_functions.default_fig()
fig = tr_functions.thesis_fig(
    title=f"EDCs of C<sub>60</sub> HOMO and HOMO-1",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    # dtick_y=0.2,
)

for i, (name, xlim) in enumerate(zip(names, xlims)):
    color = colors[i % len(colors)]

    fig1.add_vline(x=xlim[0], line_width=3, line_dash="dash", line_color=color)
    fig1.add_vline(x=xlim[1], line_width=3, line_dash="dash", line_color=color)

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_s2,
        y=y_s2,
        data=analysis_functions.norm_data(data_s2),
        ylims=None,
        x_range=xlim,
    )

    x = y_plot
    data = col

    offset_type = "constant"

    ## Offset
    # c = np.mean(data)
    c = 0
    b = (data[-1] - data[0]) / (x[-1] - x[0])
    a = 0

    offset = fitting_functions.offset_model(offset_type, a, b, c)

    ## Gaussian
    if homo_zero == True:
        gauss1 = fitting_functions.make_gaussian(
            num="A_", amplitude=1, center=0, sigma=0.5
        )
        gauss2 = fitting_functions.make_gaussian(
            num="B_", amplitude=1, center=-1.3, sigma=0.5
        )

    else:
        gauss1 = fitting_functions.make_gaussian(
            num="A_", amplitude=1, center=-2, sigma=0.5
        )
        gauss2 = fitting_functions.make_gaussian(
            num="B_", amplitude=1, center=-3.3, sigma=0.5
        )

    full_model = gauss1 + gauss2 + offset

    params = full_model.make_params()
    params["iA__center"].min = params["iA__center"].value - 0.5
    params["iA__center"].max = params["iA__center"].value + 0.5

    fit = full_model.fit(data, x=x, params=params)

    # print(f"{name} HOMO")
    # print(fit.params["iA__center"])
    # print(fit.params["iA__fwhm"])
    # print(f"{name} HOMO-1")
    # print(fit.params["iB__center"])
    # print(fit.params["iB__fwhm"])

    fig.add_trace(
        go.Scatter(
            x=y_plot,
            y=col + i * spacing,
            name=f"{name} data",
            line=dict(color=color),
            opacity=0.6,
        )
    )

    fig.add_trace(
        go.Scatter(
            x=y_plot,
            y=fit.eval(x=y_plot) + i * spacing,
            name=f"{name} fit",
            line=dict(color=color),
        )
    )

    for model_name, model_value in list(components.items())[0:2]:
        fig.add_annotation(
            x=fit.params[f"{model_name}center"].value,
            y=fit.eval(x=fit.params[f"{model_name}center"].value) + i * spacing - 0.45,
            text=f'{fit.params[f"{model_name}center"].value:.2f} eV',
            showarrow=False,
            font=dict(size=18, color=color),
            # ay=-50,
            # yref="y domain",
            # y=0.3 * i,
            bgcolor="white",
        )
        fig.add_annotation(
            x=fit.params[f"{model_name}center"].value,
            y=fit.eval(x=fit.params[f"{model_name}center"].value) + i * spacing - 0.75,
            text=f'fwhm: {fit.params[f"{model_name}fwhm"].value:.2f}',
            showarrow=False,
            font=dict(size=18, color=color),
            # ay=0,
            bgcolor="white",
        )

fig.update_layout(legend_traceorder="reversed")
fig.show()


gm = 0.375
gk = 0.43
fig1.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=[-gk, 0, gm, gm * 2, gm * 3],
        ticktext=[f"K", f"{gamma}", f"M", f"{gamma}'", f"M'"],
    ),
)

fig1.update_coloraxes(colorscale="Blues", reversescale=False)
fig1.show()

# FS (CEC)s

In [ ]:
slice_dim = "y"
int_range = 0.1
EF = 16.9

xlim = (-0.52, 0.52)
ylim = (-1.08, 0.37)
ylim = (-1.01, 0.35)

for i, v in enumerate([EF - 1.7, EF - 2.1, EF - 2.5]):
    # for i, v in enumerate([EF - 2.9, EF - 3.3, EF - 3.8]):
    ## Get data
    x_2d, y_2d, d_2d = tr_functions.slice_datacube(
        ad_homo,
        # ad_homo1,
        slice_dim,
        v,
        int_range,
        xlim,
        ylim,
        x_bin,
        y_bin,
        norm_data=True,
        plot_data=False,
    )

    fig = tr_functions.thesis_fig(
        title=f"CEC: {np.round(v - EF, 3)} eV",
        xaxis_title=f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)",
        yaxis_title=f"k<sub>y</sub> ({angstrom}<sup>-1</sup>)",
        equiv_axes=True,
        gridlines=False,
        dtick_y=0.4,
        dtick_x=0.4,
        height=500,
        width=500,
    )

    fig.add_trace(
        go.Heatmap(
            x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
        )
    )

    if xlim is not None:
        fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

    fig.update_coloraxes(colorscale="Blues", reversescale=False)

    fig.show()